In [ ]:
import cv2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import imutils
import math 

In [ ]:
from matplotlib import image
import random

In [ ]:
#note on mac or linux: change backslash direction and remove double backslash 
pathf_5x = 
pathf_10x =
pathf_20x =

In [ ]:
#import WSI
wsi_5x = cv2.imread(pathf_5x + '5x.tif')
wsi_10x = cv2.imread(pathf_10x + '10x.tif')
wsi_20x = cv2.imread(pathf_20x + '20x.tif')

In [ ]:
plt.imshow(wsi_20x)

### 1. Small crop match

### 20x 

In [ ]:
#Test matching between 5x and 20x for small cropped sections. Keep parameters associated with best match 
count = 0 
best_quality = 0 #match quality 
while count< 150: #check 100 cropped sections
    figure20, final_factor20,x,y,x20,y20, quality20, matches20 =scaleFind(wsi_5x, wsi_20x, 'SIFT', resize_factor=0.25,ratio=0.75,reprojThresh=0.5)
    if (quality20 > best_quality) and (matches20 >greatest_matches): 
        x_final = x20  
        y_final = y20
        final_factor3 = final_factor20 
        best_quality = quality20 
    count = count +1 

In [ ]:
#Save best match parameters
final_factor_20 = final_factor3
global_shiftx_20 = x_final*10    
global_shifty_20 = y_final*10  
print('*******COPY TO MATLAB********')
print('global_shiftx_20=',global_shiftx_20)
print('global_shifty_20=',global_shifty_20)
print('final_factor_20=',final_factor_20)

In [ ]:
#Visual match check 
#Transparent overlay allows you to see if images are aligned 
buffer = 200
buffer2 =100
width = wsi_5x.shape[0] +buffer 
height = wsi_5x.shape[1] +buffer 
wsi_20x_resize3 = cv2.resize(wsi_20x,(np.int(wsi_20x.shape[1]*final_factor_20),np.int(wsi_20x.shape[0]*final_factor_20)))

result1= np.zeros((width, height, 3), np.uint8)
result2= np.zeros((width, height, 3), np.uint8)

result1[buffer2 :wsi_5x.shape[0]+buffer2, buffer2:wsi_5x.shape[1]+buffer2] = wsi_5x
result2[buffer2 +np.int(global_shifty_20/10):buffer2+wsi_20x_resize3.shape[0] + np.int(global_shifty_20/10), buffer2+np.int(global_shiftx_20/10):buffer2+wsi_20x_resize3.shape[1] + np.int(global_shiftx_20/10)] = wsi_20x_resize3 

dst = cv2.addWeighted(result1,0.5,result2,0.5,0)
plt.imshow(dst)

In [ ]:
cv2.imwrite(pathf_20x +'result.tiff',dst)

## Methods

In [ ]:
def scaleFind(imgA, imgB, method, resize_factor,ratio=0.75, reprojThresh=0.5):
    #imgA is the low resolution image (5x) 
    #imgB is the higher resolution image (10x)
    #resize factor = 0.5 for 10x to 5x match 
    #height and width of cropped image 
    height_A = 600 
    width_A = 600
    
    #top left corner of crop, random start position
    random.seed(a=None, version=2)
    startx_A = random.randrange(400,imgA.shape[1]-width_A-400) 
    starty_A = random.randrange(400,imgA.shape[1]-height_A-400) 

    imgA_crop = imgA[starty_A:height_A+starty_A,startx_A:width_A+startx_A]

    #resize 10x 
    imgB_resize = cv2.resize(imgB,(np.int(imgB.shape[1]*resize_factor),np.int(imgB.shape[0]*resize_factor)))
    height_B = np.int(height_A*0.75)
    width_B = np.int(width_A*0.75)
    startx_B = startx_A  
    starty_B = starty_A  
    imgB_crop = imgB_resize[starty_B:height_B+starty_B,startx_B:width_B+startx_B]

    #First find scale [1]
    (factor, matches, status, H,theta)= returnScale(imgB_crop,imgA_crop,  'SIFT', ratio=0.75, reprojThresh=0.5)
    
    new_factor = resize_factor*factor
    imgB_resize2 = cv2.resize(imgB,(np.int(imgB.shape[1]*new_factor),np.int(imgB.shape[0]*new_factor)))
    imgB_crop = imgB_resize2[starty_B:height_B+starty_B,startx_B:width_B+startx_B]
    (factor, matches, status, H,theta)= returnScale(imgB_crop,imgA_crop,  'SIFT', ratio=0.75, reprojThresh=0.5)
    final_factor = factor*new_factor
                                    
    #x and y shift 
    x = np.int(H[0,2]) 
    y = np.int(H[1,2]) 
    
    x2 = H[0,2]
    y2 = H[1,2] 

    #stitch resized image 
    figure_stitch = stitchMosaics(imgA_crop, imgB_crop, x, y)
#     plt.show()
#     plt.imshow(figure_stitch)

    quality = np.sum(status)/len(matches)                            
    return figure_stitch, final_factor,x,y,x2,y2, quality, len(matches)

In [ ]:
def returnScale(imgA, imgB, method, ratio=0.75, reprojThresh=1.0):
    (kpsA, featuresA) = detectAndDescribe(imgA)
    (kpsB, featuresB) = detectAndDescribe(imgB)

    matcher = cv2.DescriptorMatcher_create("BruteForce")

    # perform K-NN matching between the 2 feature vector sets, using k= 2
    rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
    matches = []

    # loop over the raw matches to prune false positives
    for m in rawMatches:
        # ensure the distance is within a certain ratio of each
        # other (i.e. Lowe's ratio test)
        if len(m) == 2 and m[0].distance < m[1].distance * ratio:
            # item.trainIdx: This attribute gives us the index of the descriptor in the list of train descriptors (the list of descriptors in the img2).
            # item.queryIdx: This attribute gives us the index of the descriptor in the list of query descriptors (the list of descriptors in the img1).
            matches.append((m[0].trainIdx, m[0].queryIdx))

    if len(matches) > 4:
        # construct the two sets of points
        ptsA = np.float32([kpsA[i] for (_, i) in matches])
        ptsB = np.float32([kpsB[i] for (i, _) in matches])

    (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

    a = H[0, 0]
    b = H[0, 1]
    d = H[1, 0]
    e = H[1, 1]
    p = math.sqrt(a ** 2 + b ** 2)
    factor = (a * e + b * d) / p
    theta = math.atan2(H[0,1],H[0,0])*180/math.pi

    return factor, matches, status, H, theta

def detectAndDescribe(image):
    """detects keypoints and extracts local invariant descriptors"""

    descriptor = cv2.xfeatures2d.SIFT_create()

    # detectAndCompute -- extracts keypoints and features, Detects keypoints and computes the descriptors
    (kps, features) = descriptor.detectAndCompute(image, None)

    if features is None or kps is None:
        return (None, None)

    else:
        kps = np.float32([kp.pt for kp in kps])
        return (kps, features)
    
def stitchMosaics(imageA, imageB, x, y):
    width = imageA.shape[0] +100
    height = imageA.shape[1] +100

    result_stitch = np.zeros((width, height, 3), np.uint8)
    shiftx = x+50
    shifty = y+50

    result_stitch[50:imageA.shape[0]+50, 50:imageA.shape[1]+50] = imageA
    roi = result_stitch[shifty:imageB.shape[0] + shifty, shiftx:imageB.shape[1] + shiftx]
    # Now create a mask of logo and create its inverse mask also
    img2gray = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Now black-out the area of logo in ROI
    img1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

    img2_fg = cv2.bitwise_and(imageB, imageB, mask=mask)

    # Put logo in ROI and modify the main image
    dst = cv2.add(img1_bg, img2_fg)
    result_stitch[shifty:imageB.shape[0] + shifty, shiftx:imageB.shape[1] + shiftx] = dst

    # return the stitched image
    return result_stitch